In [8]:
#Max Shi CS 559 HW 1 Problem 5

import pandas as pd
df = pd.read_excel("ENB2012_data.xlsx", 0)
print(df)

       X1     X2     X3      X4   X5  X6   X7  X8     Y2
0    0.98  514.5  294.0  110.25  7.0   2  0.0   0  21.33
1    0.98  514.5  294.0  110.25  7.0   3  0.0   0  21.33
2    0.98  514.5  294.0  110.25  7.0   4  0.0   0  21.33
3    0.98  514.5  294.0  110.25  7.0   5  0.0   0  21.33
4    0.90  563.5  318.5  122.50  7.0   2  0.0   0  28.28
..    ...    ...    ...     ...  ...  ..  ...  ..    ...
763  0.64  784.0  343.0  220.50  3.5   5  0.4   5  21.40
764  0.62  808.5  367.5  220.50  3.5   2  0.4   5  16.88
765  0.62  808.5  367.5  220.50  3.5   3  0.4   5  17.11
766  0.62  808.5  367.5  220.50  3.5   4  0.4   5  16.61
767  0.62  808.5  367.5  220.50  3.5   5  0.4   5  16.03

[768 rows x 9 columns]


In [9]:
from sklearn import linear_model

In [10]:
features = df[df.columns[0:8]].values
result = df[df.columns[-1]].values


In [11]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(features, result)
print(clf.coef_)


[-0.          0.01259381  0.02982036 -0.          5.16871044  0.04151042
  8.91526652  0.10520176]


In [12]:
reg = linear_model.Ridge(alpha=0.5)
reg.fit(features, result)
print(reg.coef_)

[-1.00031000e+01  1.01945613e-02  2.01912798e-02 -4.99835887e-03
  5.53880152e+00  1.21447163e-01  1.41718290e+01  5.06574211e-02]


In [13]:
net = linear_model.ElasticNet(alpha=0.5)
net.fit(features, result)
print(net.coef_)

[-0.         -0.04426558  0.08612412 -0.01621481  1.7237392   0.
  0.01666456  0.18616842]


In [14]:
#method to create a model with the given model_name from the given data
def train(features, values, model_name):
    model = None
    if(model_name == "Lasso"):
        model = linear_model.Lasso(alpha=0.1)
    elif(model_name == "Ridge"):
        model = linear_model.Ridge(alpha=0.1)
    else:
        model = linear_model.ElasticNet(alpha=0.1)
    model.fit(features,values)
    return model


In [15]:
#calculate the validation error from the model, the test data, and the loss function
def validation_error(model, test_features, test_values, loss_func):
    predictions = model.predict(test_features)
    error = 0
    for model_prediction, test in zip(predictions, test_values):
        error += loss_func(model_prediction, test)
    return error / len(predictions)

In [16]:
#MSE Loss function
def MSE_loss(prediction, test):
    return pow(prediction-test, 2)

In [24]:
#function to do x-fold cross validation with a given model name and features and values data
def cross_validation(folds, model, features, values):
    feature_cuts = []
    values_cuts = []
    interval = (len(features) // folds)+1
    for x in range(folds):
        feature_cuts.append(features[x*interval:(x+1)*interval])
        values_cuts.append(values[x*interval:(x+1)*interval])
    errors = []
    for x in range(folds):
        training_features = []
        training_values = []
        for y in range(folds):
            if x != y:
                #print(feature_cuts[y])
                training_features.extend(feature_cuts[y])
                training_values.extend(values_cuts[y])
        #print(len(training_features), len(training_values))
        trained_model = train(training_features, training_values, model)
        #print(trained_model)
        errors.append(validation_error(trained_model, feature_cuts[x], values_cuts[x], MSE_loss))
    print(r"model %s" % model, errors)
    return sum(errors)/len(errors)


In [26]:
#do the cross validations.
model_names = ["Lasso", "Ridge", "ElasticNet"]
print("model and average MSE error")
list(map(lambda model_name: (model_name, cross_validation(5, model_name, features,result)), model_names))


model and average MSE error
model Lasso [22.749548468453607, 12.748086474949778, 10.333382321636469, 13.36013496908495, 10.36050038188607]
model Ridge [12.853088671889545, 12.031837023054782, 9.961008783459354, 10.348433956734526, 10.35542816063348]
model ElasticNet [29.36383974550231, 17.040449290138064, 10.721510707794405, 17.395113877288228, 13.713624107934328]


[('Lasso', 13.910330523202173),
 ('Ridge', 11.109959319154338),
 ('ElasticNet', 17.646907545731466)]

In [ ]:
#Based on the MSE error seen in the cross validation between the models, we conclude that the Ridge model
#seems to be more effective to predict cooling load when alpha values are equal across all models.